In [36]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential, Model
import pickle
import matplotlib.pyplot as plt
import os
import db_connection as db



In [37]:
# ---------- 1. Data Loading & Preprocessing ----------
# Load cleaned flight data
df = db.read_db("flights_cleaned")
df.columns.tolist()
df.head()


,latitude,longitude,gps_altitude_m,distance_m,speed_km/s,climb_m,climb_m(delta),climb_rate_m/s,glide_ratio,bearing,delta_bearing,elapsed_time,delta_time,temp,pressure,humidity,dew_point,wind_speed,wind_deg
0,36.980983,29.314417,2083,7.545830,27.164986,-2.0,0.0,-2.0,3.772915,11,3.0,17.0,1.0,25.62,1008.0,35.0,9.04,0.66,75.0
1,36.981050,29.314450,2082,7.971536,28.697530,-3.0,0.0,-3.0,2.657179,21,10.0,18.0,1.0,25.62,1008.0,35.0,9.04,0.66,75.0
2,36.981150,29.314500,2080,11.957302,43.046289,-2.0,0.0,-2.0,5.978651,21,0.0,19.0,1.0,25.62,1008.0,35.0,9.04,0.66,75.0
3,36.981217,29.314567,2079,9.485179,34.146645,-1.0,-14.0,-1.0,9.485179,38,17.0,20.0,1.0,25.62,1008.0,35.0,9.04,0.66,75.0
4,36.981283,29.314633,2078,9.485176,34.146634,0.0,-14.0,0.0,0.000000,38,0.0,21.0,1.0,25.62,1008.0,35.0,9.04,0.66,75.0


In [38]:
# Separate features and target
feature_cols = [
    'gps_altitude_m', 'distance_m', 'speed_km/s', #'climb_m', 
    'glide_ratio', 'bearing', 'delta_bearing',
    'temp', 'pressure', 'humidity', 'dew_point',
    'wind_speed', 'wind_deg'
]
X_raw = df[feature_cols].values.astype(np.float32)
y_raw = df['climb_rate_m/s'].values.astype(np.float32)

# Train-test split on raw data
split_frac = 0.8
split_idx = int(len(X_raw) * split_frac)
X_train_raw, X_test_raw = X_raw[:split_idx], X_raw[split_idx:]
y_train_raw, y_test_raw = y_raw[:split_idx], y_raw[split_idx:]

# Feature scaling
scaler = StandardScaler()
scaler.fit(X_train_raw)
X_train_scaled = scaler.transform(X_train_raw)
X_test_scaled  = scaler.transform(X_test_raw)

# Persist scaler locally
os.makedirs('models', exist_ok=True)
scaler_path = os.path.join('models', 'scaler.pkl')
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)
print(f"Scaler saved to {scaler_path}")


Scaler saved to models\scaler.pkl


In [39]:

# ---------- 2. Create tf.data sliding-window datasets ----------
T = 10              # history length
batch_size = 32
train_ds = tf.keras.preprocessing.timeseries_dataset_from_array(
    data=X_train_scaled,
    targets=y_train_raw,
    sequence_length=T,
    sequence_stride=1,
    shuffle=False,
    batch_size=batch_size
)
test_ds = tf.keras.preprocessing.timeseries_dataset_from_array(
    data=X_test_scaled,
    targets=y_test_raw,
    sequence_length=T,
    sequence_stride=1,
    shuffle=False,
    batch_size=batch_size
)

# ---------- 3. Positional Encoding for Transformer ----------
def get_positional_encoding(sequence_length, d_model):
    pos = np.arange(sequence_length)[:, np.newaxis]
    i   = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / d_model)
    angle_rads  = pos * angle_rates
    sines = np.sin(angle_rads[:, 0::2])
    cosines = np.cos(angle_rads[:, 1::2])
    return tf.cast(np.concatenate([sines, cosines], axis=-1), tf.float32)

# Dimensions
n_features = X_raw.shape[1]

def build_lstm_model(seq_len, n_feats):
    inp = layers.Input(shape=(seq_len, n_feats))
    x = layers.LSTM(64)(inp)
    x = layers.Dense(32, activation='relu')(x)
    out = layers.Dense(1)(x)
    return Model(inp, out, name='LSTM')


def build_rnn_model(seq_len, n_feats):
    inp = layers.Input(shape=(seq_len, n_feats))
    x = layers.SimpleRNN(64)(inp)
    x = layers.Dense(32, activation='relu')(x)
    out = layers.Dense(1)(x)
    return Model(inp, out, name='RNN')


def build_cnn_model(seq_len, n_feats):
    inp = layers.Input(shape=(seq_len, n_feats))
    x = layers.Conv1D(32, 3, activation='relu')(inp)
    x = layers.MaxPooling1D(2)(x)
    x = layers.Conv1D(64, 3, activation='relu')(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(32, activation='relu')(x)
    out = layers.Dense(1)(x)
    return Model(inp, out, name='CNN')


def build_transformer_model(seq_len, n_feats, d_model=64, num_heads=4):
    inp = layers.Input(shape=(seq_len, n_feats))
    x = layers.Dense(d_model)(inp)
    pos_enc = get_positional_encoding(seq_len, d_model)
    x = x + pos_enc
    attn = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(x, x)
    x = layers.LayerNormalization(epsilon=1e-6)(x + attn)
    ffn_block = Sequential([
        layers.Dense(d_model*2, activation='relu'),
        layers.Dense(d_model)
    ])
    x = layers.LayerNormalization(epsilon=1e-6)(x + ffn_block(x))
    x = layers.GlobalAveragePooling1D()(x)
    out = layers.Dense(1)(x)
    return Model(inp, out, name='Transformer')

# Map model names to builders
type_map = {
    'LSTM': build_lstm_model,
    'RNN': build_rnn_model,
    'CNN': build_cnn_model,
    'Transformer': build_transformer_model
}


In [ ]:

# ---------- 4. Training Loop & Saving ----------
results = {}
for name, builder in type_map.items():
    print(f"\n>>> Training {name} <<<")
    model = builder(T, n_features)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    model.fit(train_ds, validation_data=test_ds, epochs=1, verbose=1)
    # Evaluate
    y_pred = model.predict(test_ds).flatten()
    y_true = np.concatenate([y for _, y in test_ds], axis=0)
    mae = np.mean(np.abs(y_true - y_pred))
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    results[name] = (mae, rmse)
    # Save
    save_path = os.path.join('models', f"{name.lower()}_model.keras")
    model.save(save_path)
    print(f"{name} saved to {save_path}")



>>> Training LSTM <<<


In [ ]:

# ---------- 5. Summary ----------
print("\n=== Test Metrics ===")
for name, (mae, rmse) in results.items():
    print(f"{name}: MAE={mae:.4f}, RMSE={rmse:.4f}")


In [ ]:

# ---------- 6. Plot error histograms ----------
for name in type_map.keys():
    model = tf.keras.models.load_model(os.path.join('models', f"{name.lower()}_model.keras"))
    y_pred = model.predict(test_ds).flatten()
    y_true = np.concatenate([y for _, y in test_ds], axis=0)
    errors = y_true - y_pred
    plt.figure()
    plt.hist(errors, bins=50)
    plt.title(f"{name} Error Histogram")
    plt.xlabel('Error (m/s)')
    plt.ylabel('Count')
    plt.show()
